# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа в отрезке [a, b] с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))


def getrandbits(n) -> gmpy2.mpz:
    """
    Генерация случайного числа длиной n бит с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 13259308990068677248458814546655642110686212163633534988815383338310479145526660581908639346320465565095161582896480982141993175548326119533240579155396635685701737031626124288999159490935947720681151107749301472952873777982216656569913096012114126690660837033408733761519379711611293873626825973892012923032312569443626466347612297171229597716722821508722502415605404689774248571730075956992596265415052876739582826902573365792946376165477155331756837294077711080013406998125583890170608021380229162301382539492944261965689663319182347351092647149409244122953896278941862427224960421938840768606322399500400220518777


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client 0ee06dea-b96e-45c5-9beb-7d6f7d7764d3
[TRACE] [FiatShamirClient] Public key = 10361570848189380083744207830969150117573113129587314452254405521415791426065325960357539526741064458129436446009748019681022204557979377534231949665779760983282334792962674635862543806053686444086849470583096677362481613737021884164106765192787905303925085042675382485027534673787034802885941957854339706511243357094172967379470849993203602451824159854827225253842320687784278649414726076567826809417005261250576186133274210810482734210275789948016139023134997328772371726390926224145186653834427742419482300958418373735645880606512980925777049025418218085293490332163024966002058348139783101769936372966738535949971
[TRACE] [FiatShamirClient] Client 0ee06dea-b96e-45c5-9beb-7d6f7d7764d3
[TRACE] [FiatShamirClient] Public key = 1036157084818938008374420783096915011757311312958731445225440552141579142606532596035753952674106445812943644600974801968102220455797937753423194966577976098328

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client 0ee06dea-b96e-45c5-9beb-7d6f7d7764d3 
        with public key 10361570848189380083744207830969150117573113129587314452254405521415791426065325960357539526741064458129436446009748019681022204557979377534231949665779760983282334792962674635862543806053686444086849470583096677362481613737021884164106765192787905303925085042675382485027534673787034802885941957854339706511243357094172967379470849993203602451824159854827225253842320687784278649414726076567826809417005261250576186133274210810482734210275789948016139023134997328772371726390926224145186653834427742419482300958418373735645880606512980925777049025418218085293490332163024966002058348139783101769936372966738535949971
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 0ee06dea-b96e-45c5-9beb-7d6f7d7764d3
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(12657467969693482410728961634793573017715180097016059079982173997510874143344414431601288673209436419098442456148210383291325844587921121541525350664796310904807334597032108074093594713756445612197742023667842082362051776071447906221208581848784249754126465449041649333914775886093343224736076073736479107401831593115099432576780523586895686844803643607121684755910736699518018176112231583414612270370620579911893245822499629100807960862334015105027559968806404325865869048260103226757966345740384801964986917240146435485334623032070000157124816374939070784332781656413237063057307866506999564030900463624466863215282)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(0)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(37609814050548030177858456633362888322994383668703843783476910733014627104

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 0ee06dea-b96e-45c5-9beb-7d6f7d7764d3
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(6843340469725185686533452507313385102968373259328392856317959038900121168823077344574657776185971065377811103855849565711407765007490124328836825692044368899863381044389539309184960300328791938806848184887823539822342805959760440958832053400176730692370211953804184904541187485259786074549960832989271293483712633543069775176733566782004920256828446553301417470863752440584986659309325553986587293950811827022021256164778753826964281233429084539140991130625194943433892124271190317410633612550415163768799460224822447115592775978655063419149411040959214051197532434697442455448733157478020993143923103800553094164388)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(0)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(386985719055994335656795954369626819701016786339854924151323607004330052845

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, 2 ** self._security_parameter)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        is_successful = x == z
        print(f'Authentication successful: {is_successful}')
        
        return is_successful

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(88719642581720226840481950021590303374356181890516660980145883473215432582094142205336455039150207482291110059881154078977881393080847402264681691893467562090402584942296739914867448816063863695644322215943398522131210259006237314444192688888601348141273996991865931780829162097458873815537047988319521147147), q = mpz(920978127834024010539279593936512627918654157983), g = mpz(24726592086968156803226812371248703422295398127599303184890432087511977926449403377304843768181406578411862525099597243117257923322395575871673176710951064152621008436388295762630689368763488833954650320056964277204707497766075464905500461667204447376951520147744892092302136854379501337440539134020456251520)
[TRACE] [SchnorrCA] RSA public key = (65537, 66015934736521896933066117295568382438004969348100901660421684412592650401485718825606902091830965547333025564122796255026667257099733539920263135661481111830086349331965480664687146394396496883197702337527892223941458330454591587768118696

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client 793de803-d85d-41fa-ae47-b3a7222597f4
[TRACE] [SchnorrClient] Public key = 32220610832229358200657099382799418543906378080511131810339831083119730568554276760692509162658205655188265080354671464502320757537993274732141825473718547177819301901771541049847477403197438335696569947511681579281298594563721793738296066665173368902009841448437461561439923793762074679296341956539340828462


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('793de803-d85d-41fa-ae47-b3a7222597f4'), mpz(32220610832229358200657099382799418543906378080511131810339831083119730568554276760692509162658205655188265080354671464502320757537993274732141825473718547177819301901771541049847477403197438335696569947511681579281298594563721793738296066665173368902009841448437461561439923793762074679296341956539340828462), b'\x976%\xc6K!\xce<\xa1\x916\xbc\x91\xaa\x8a\x1b\xa6U\xef}\xad\x1e\x82\xf5\x9a\xae(\n\xde\xf67\xc0[\x93\xaf!\xc3\xf1\x10\x03s"Y\xda\xa7CPL\x95\xef]\xf5P\x8d\x85w\xe3\xae`\x9f\x0b\x93v!36A\x99;?;.\r\xf3\x1a\xcd\x16\x9c\xc8\xf9QON\x9d\xb1=^\x17{\x90\xc9\xb4\x83\xe1@kb\x1e\x00\x81\xf4D\xef\xc6\x06\x1b\x07\xfc\xfbI\x04\x01\xf7\x06\xc7u\xc1\xd2\x9d\xe8A]\xe8\xf5}\x16\x9d\xd0\xdd\xa1\x9b*\x0e\xd6Z\xc0\xb9\x97\xcf\x17\xe4YYky\xc8oe\xc5\x96\nJ\xd5\xfd\xcc\xaa\xfdf\x10\xc6\x91@\x7f\n6\x98\xf9\xfe\xdc\xcc\xc2>\xed\xb4\x11/\xcfN\xc4\xa8\xee\xc7\xf5\xdd:\xc4\x90Y\x10\xe4\x00`\xf9rF\xad\x15w\x02G\x16\xb7\x82\x

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(27636347566407772312485417743231837832418388199065989124180328903005724345149866361742799224521217733881996060852218354047661025087364000577577405230038632011220973872846453578541786099415315432929836741655520096228166772889941241473633599092163625798741356077801847678366070900658287599439469647774409695262)
[TRACE] [SchnorrVerifier] e = mpz(183)
[TRACE] [SchnorrVerifier] y = mpz(811024823003081569183321041443365768810230755886)
[TRACE] [SchnorrVerifier] z = mpz(27636347566407772312485417743231837832418388199065989124180328903005724345149866361742799224521217733881996060852218354047661025087364000577577405230038632011220973872846453578541786099415315432929836741655520096228166772889941241473633599092163625798741356077801847678366070900658287599439469647774409695262)
Authentication successful: True


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client 793de803-d85d-41fa-ae47-b3a7222597f4
[TRACE] [SchnorrClient] Public key = 32220610832229358200657099382799418543906378080511131810339831083119730568554276760692509162658205655188265080354671464502320757537993274732141825473718547177819301901771541049847477403197438335696569947511681579281298594563721793738296066665173368902009841448437461561439923793762074679296341956539340828462


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(43456842040202530125713502751000351469506197532263435283638696013176525939862157385999671456156179475090740359784771848096207980269014659240846811868359377133416359618912700308925401489439417988003325889638879806093387451842471673157413980097325270582381636014494015384080479975169140451515103692066542542231)
[TRACE] [SchnorrVerifier] e = mpz(209)
[TRACE] [SchnorrVerifier] y = mpz(466953623065483343815966175423320171420701493919)
[TRACE] [SchnorrVerifier] z = mpz(24648151552571952676184165103099668870630895761404428501067840164327662723786975114707388696705713385699949467982824528779926822033032065255468029938503509885441823191671160661793528285755117195893832486171102927725205203697949660411878362259450235165254012248123428622271670111071708702756072552575045478478)
Authentication successful: False


False